#### Import dependencies

In [ ]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# Importing functions from the modules in the qseg package
from qseg.graph_utils import image_to_grid_graph, draw, draw_graph_cut_edges
from qseg.dwave_utils import dwave_solver, annealer_solver
from qseg.utils import decode_binary_string

# Additional necessary imports
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from qiskit_optimization.applications import Maxcut
import dimod
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

In [ ]:
import gurobipy as gp
from gurobipy import GRB

#### Generate Problem Instance

In [ ]:
def get_nx_e_list(h, w):
    nx_elist = []
    for i in range(h*w):
        x, y = i//w, i%w
        if x > 0:
            nx_elist.append(((x,y),(x-1,y)))
        if y > 0:
            nx_elist.append(((x,y),(x,y-1)))
    return nx_elist

In [ ]:
def generate_problem_instance(height,width):
    nx_elist = get_nx_e_list(height,width)
    G = nx.grid_2d_graph(height,width)
    edge_weights = np.random.uniform(low=-1, high=1, size=len(nx_elist))
    G.add_weighted_edges_from([(node[0],node[1], w) for node,w in zip(nx_elist, edge_weights)])
    return G

#### Gurobi QUBO solver

In [ ]:
def gurobi_qubo_solver(G):
  w = -1 * nx.adjacency_matrix(G).todense()
  max_cut = Maxcut(w)
  qp = max_cut.to_quadratic_program()
  linear = qp.objective.linear.coefficients.toarray(order=None, out=None)
  quadratic = qp.objective.quadratic.coefficients.toarray(order=None, out=None)

  linear = {int(idx):-round(value,2) for idx,value in enumerate(linear[0])}
  quadratic = {(int(iy),int(ix)):-quadratic[iy, ix] for iy, ix in np.ndindex(quadratic.shape) if iy<ix and abs(quadratic[iy, ix])!=0}

  qubo_matrix = np.zeros([len(linear),len(linear)])
  for key,value in linear.items():
    qubo_matrix[int(key),int(key)] = value
  for key,value in quadratic.items():
    qubo_matrix[int(key[0]),int(key[1])] = value/2
    qubo_matrix[int(key[1]),int(key[0])] = value/2

  n = qubo_matrix.shape[0]
  model = gp.Model()
  x = model.addVars(n, vtype=GRB.BINARY)
  obj_expr = gp.quicksum(qubo_matrix[i, j] * x[i] * x[j] for i in range(n) for j in range(n))
  model.setObjective(obj_expr)
  model.setParam('OutputFlag', 0)
  model.optimize()

  if model.status == GRB.OPTIMAL:
    solution = [int(x[i].X) for i in range(n)]
    binary_string = ''.join(str(bit) for bit in solution)
    return binary_string, model.objVal
  else:
    return None, None

#### Experimental settings

In [ ]:
# seeds = [111,222,333,444,555]

seed = 111

dwave_annealer_solver = True#@param {type:"boolean"}
gurobipy_qubo_solver = True#@param {type:"boolean"}

solver_flags = ''.join(map(str,map(int,[dwave_annealer_solver, gurobipy_qubo_solver])))


table_contents = []

heights = np.arange(2, 51).tolist()
report_filename = 'imgSeg_paper_report_' + solver_flags + '_' +  str(seed) + '.txt'
qputimes_filename = 'imgSeg_paper_qputimes_' + solver_flags + '_' +  str(seed) + '.csv'

qpu_output_path = os.path.join('results', f'{seed}')



In [ ]:
# Create the directory if it doesn't exist
os.makedirs(qpu_output_path, exist_ok=True)

fieldnames = ['qpu_sampling_time', 'qpu_anneal_time_per_sample', 'qpu_readout_time_per_sample', 'qpu_access_time', 'qpu_access_overhead_time', 'qpu_programming_time', 'qpu_delay_time_per_sample', 'post_processing_overhead_time', 'total_post_processing_time', 'problem_formulation_time', 'connection_time', 'embedding_time', 'response_time', 'sample_fetch_time', 'Height', 'Width']
if dwave_annealer_solver and not os.path.exists(qputimes_filename):
  with open(qputimes_filename, mode='w', newline='') as qputime_file:
        writer = csv.DictWriter(qputime_file, fieldnames=fieldnames)
        # Write the header row
        writer.writeheader()

#### Executions

In [ ]:
for height in heights:
  for width in [height]:
    print(f'seed: {seed}, height: {height}, width: {width}, n: {height*width}')
    np.random.seed(seed=seed)
    G = generate_problem_instance_v3(height, width)

    start_time = time.time()
    if dwave_annealer_solver:
      sample_set_df, info_dict = annealer_solver(G, n_samples = n_samples)
      dwave_annealer_tte = (time.time() - start_time)
      print('dwave_annealer_tte',dwave_annealer_tte)
      dwave_annealer_solution_first = ''.join(sample_set_df.iloc[0][:-3].astype(int).astype(str))
      dwave_annealer_solution_lowest = ''.join(sample_set_df.sort_values(by=['num_occurrences', 'energy'], ascending=[False, True]).iloc[0][:-3].astype(int).astype(str))

      dwave_annealer_value_first = sample_set_df.iloc[0][-2:-1][0]
      dwave_annealer_value_lowest = sample_set_df.sort_values(by=['num_occurrences', 'energy'], ascending=[False, True]).iloc[0][-2:-1][0]

      print('dwave_annealer_value_lowest_energy',dwave_annealer_value_first)
      print('dwave_annealer_value_most_occurred',dwave_annealer_value_lowest)

      print("ANNEALER DONE!")
      qputime_dic = info_dict.copy()
      qputime_dic['Height'] = height
      qputime_dic['Width'] = width
      print('qputime_dic',qputime_dic)

      sample_set_df.to_csv(os.path.join(qpu_output_path, f'{height}_{width}.csv'), index=False)

      with open(qputimes_filename, mode='a', newline='') as qpu_times_file:
        writer = csv.DictWriter(qpu_times_file, fieldnames=qputime_dic.keys())
        writer.writerow(qputime_dic)
    else:
      dwave_annealer_solution, dwave_annealer_value = None, None


    start_time = time.time()
    if gurobipy_qubo_solver:
      try:
        gurobi_qubo_solution, gurobi_qubo_value = gurobi_qubo_solver(G)
        gurobi_qubo_tte = (time.time() - start_time)
        print('gurobi_qubo_value',gurobi_qubo_value)
        print('gurobi_qubo_tte:',gurobi_qubo_tte)
        print("GUROBI QUBO DONE!")
      except Exception as e:
        print("Gurobi QUBO execution is unsuccessful due to "+ str(e))
        gurobi_qubo_solution, gurobi_qubo_value = None, None
    else:
      gurobi_qubo_solution, gurobi_qubo_value = None, None


    row = []

    row.append(int(G.number_of_nodes()))
    row.append(int(height))
    row.append(int(width))

    if dwave_annealer_solver:
      row.append(''.join(map(str, map(int, (dwave_annealer_solution_first)))))
      row.append(np.round(dwave_annealer_value_first,4))
      row.append(''.join(map(str, map(int, (dwave_annealer_solution_lowest)))))
      row.append(np.round(dwave_annealer_value_lowest,4))
      row.append(np.round(dwave_annealer_tte,4))

    if gurobipy_qubo_solver:
      if gurobi_qubo_solution:
        row.append(''.join(map(str, map(int, (gurobi_qubo_solution)))))
      else:
        row.append(''.join(['0']*G.number_of_nodes()))
      if gurobi_qubo_value:
        row.append(np.round(gurobi_qubo_value,4))
      else:
        row.append(0)
      if gurobi_qubo_tte:
        row.append(np.round(gurobi_qubo_tte,4))
      else:
        row.append(0)

    report_file_obj = open(os.path.join(report_filename),'a+')
    report_file_obj.write('__'.join(map(str,row))+'\n')
    report_file_obj.close()
    table_contents.append(row)
    print('\n')